In [1]:
# load model

from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
    )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-02-06 10:48:06.548656: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-06 10:48:06.565156: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738806486.582608 2833459 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738806486.587950 2833459 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-06 10:48:06.610204: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Unsloth: OpenAI failed to import - ignoring for now.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: NVIDIA A100-PCIE-40GB. Max memory: 39.394 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
[
    alpaca_prompt.format(
        "Recommend a daily diet that includes a specific ingredient.",

        "Create a diet that includes baby bok choy(roots trimmed and roughly chopped).",

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Recommend a daily diet that includes a specific ingredient.

### Input:
Create a diet that includes baby bok choy(roots trimmed and roughly chopped).

### Response:


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


<|end_of_text|>


In [3]:
# load dataset

import json
import pandas as pd

file_path = "/data/jaesung/llm_for_diabetes/src/model/inference_results_test.jsonl"

data = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

df = pd.DataFrame(data)

In [4]:
from datasets import Dataset, DatasetDict

columns_to_keep = ["dataset", "split_data", "task", "instruction", "input", "output", "text_length"]
df = df[columns_to_keep]

task_grouped_data = {task: df[df["task"] == task] for task in df["task"].unique()}

sampled_dataset = DatasetDict({task: Dataset.from_pandas(task_df, preserve_index=False) for task, task_df in task_grouped_data.items()})

sampled_dataset


DatasetDict({
    daily_diets: Dataset({
        features: ['dataset', 'split_data', 'task', 'instruction', 'input', 'output', 'text_length'],
        num_rows: 25
    })
    generation: Dataset({
        features: ['dataset', 'split_data', 'task', 'instruction', 'input', 'output', 'text_length'],
        num_rows: 25
    })
    alternative_diet: Dataset({
        features: ['dataset', 'split_data', 'task', 'instruction', 'input', 'output', 'text_length'],
        num_rows: 25
    })
    summarization: Dataset({
        features: ['dataset', 'split_data', 'task', 'instruction', 'input', 'output', 'text_length'],
        num_rows: 25
    })
    re: Dataset({
        features: ['dataset', 'split_data', 'task', 'instruction', 'input', 'output', 'text_length'],
        num_rows: 25
    })
    qa1: Dataset({
        features: ['dataset', 'split_data', 'task', 'instruction', 'input', 'output', 'text_length'],
        num_rows: 25
    })
    qa2: Dataset({
        features: ['dataset', 'split

In [5]:
import json
import re
from tqdm import tqdm
from transformers import TextStreamer

FastLanguageModel.for_inference(model)

output_file = "inference_results_zero_shot.jsonl"


task_max_new_tokens = {
    "alternative_diet": 512,
    "daily_diets": 1024,
    "dfh_info": 256,
    "generation": 2048,
    "ie": 128,
    "nli": 32,
    "qa_objective_1": 64,
    "qa_objective_2": 64,
    "qa_objective_3": 64,
    "re": 64,
    "summarization": 1024, 
}


# 1. `tqdm`을 사용하여 진행 상태 표시
total_samples = sum(len(samples) for samples in sampled_dataset.values())
with open(output_file, 'w') as f_out:
    with tqdm(total=total_samples, desc="Processing samples", unit="sample") as pbar:
        for task, sample in sampled_dataset.items():
            for samp in sample:
                # 데이터셋 이름에 따른 context length 설정
                task_name = samp.get("task", "")
                # max_new_tokens = 8192 if dataset_name in ["meddialog", "pubmed"] else 8
                max_new_tokens = task_max_new_tokens.get(task_name, 8)  # 기본값 8


                # 입력 토큰 생성
                inputs = tokenizer(
                    [
                        alpaca_prompt.format(
                            samp['instruction'],  # instruction
                            samp['input'],  # input
                            "",  # output
                        )
                    ], return_tensors="pt"
                ).to("cuda")

                # TextStreamer 설정
                text_streamer = TextStreamer(tokenizer)
                
                # 모델 생성 및 출력
                output_tensor = model.generate(
                    **inputs, 
                    max_new_tokens=max_new_tokens
                )
                model_output = tokenizer.decode(output_tensor[0], skip_special_tokens=True)

                # `### Response:` 뒤의 텍스트 추출
                response_text = None
                response_match = re.search(r"### Response:\s*(.+)", model_output, re.DOTALL)
                if response_match:
                    response_text = response_match.group(1).strip()
                else:
                    response_text = "No valid response found"

                # 모델 출력 결과를 samp에 추가
                samp['model_output'] = response_text

                # JSONL 형식으로 저장
                f_out.write(json.dumps(samp, ensure_ascii=False) + "\n")
                
                # tqdm 진행 상태 업데이트
                pbar.update(1)


Processing samples: 100%|██████████| 250/250 [29:21<00:00,  7.05s/sample] 


In [6]:
import json
import pandas as pd

file_path = "/data/jaesung/llm_for_diabetes/src/model/inference_results_zero_shot.jsonl"

data = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

df = pd.DataFrame(data)

In [7]:
# medqa

import pandas as pd
qa_objective_df = df[df['task']=='qa1']

# 'output'와 'model_output'에서 A), B), C), D)만 추출
qa_objective_df['output_label'] = qa_objective_df['output'].str.extract(r'(A\)|B\)|C\)|D\))')
qa_objective_df['model_output_label'] = qa_objective_df['model_output'].str.extract(r'(A\)|B\)|C\)|D\))')

# 두 컬럼 비교하여 맞은 경우를 계산
qa_objective_df['correct'] = qa_objective_df['output_label'] == qa_objective_df['model_output_label']

# Accuracy 계산
accuracy = qa_objective_df['correct'].mean()

print(f"Accuracy: {accuracy:.2%}")


Accuracy: 4.00%


/tmp/ipykernel_2833459/1017709785.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_objective_df['output_label'] = qa_objective_df['output'].str.extract(r'(A\)|B\)|C\)|D\))')
/tmp/ipykernel_2833459/1017709785.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_objective_df['model_output_label'] = qa_objective_df['model_output'].str.extract(r'(A\)|B\)|C\)|D\))')
/tmp/ipykernel_2833459/1017709785.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [8]:
# medmcqa

import pandas as pd
qa_subjective_df = df[df['task']=='qa2']

# 'output'와 'model_output'에서 A), B), C), D)만 추출
qa_subjective_df['output_label'] = qa_subjective_df['output'].str.extract(r'(A\)|B\)|C\)|D\))')
qa_subjective_df['model_output_label'] = qa_subjective_df['model_output'].str.extract(r'(A\)|B\)|C\)|D\))')

# 두 컬럼 비교하여 맞은 경우를 계산
qa_subjective_df['correct'] = qa_subjective_df['output_label'] == qa_subjective_df['model_output_label']

# Accuracy 계산
accuracy = qa_subjective_df['correct'].mean()

print(f"Accuracy: {accuracy:.2%}")

Accuracy: 0.00%


/tmp/ipykernel_2833459/2145043051.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_subjective_df['output_label'] = qa_subjective_df['output'].str.extract(r'(A\)|B\)|C\)|D\))')
/tmp/ipykernel_2833459/2145043051.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_subjective_df['model_output_label'] = qa_subjective_df['model_output'].str.extract(r'(A\)|B\)|C\)|D\))')
/tmp/ipykernel_2833459/2145043051.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [9]:
# pubmedqa

def token_overlap(output, model_output):
    # 토큰화
    output_tokens = set(output.split())
    model_output_tokens = set(model_output.split())
    
    # 공통 토큰 개수 계산
    common_tokens = output_tokens.intersection(model_output_tokens)
    
    # 일치 비율 계산
    return len(common_tokens) / len(output_tokens) if len(output_tokens) > 0 else 0

qa_descriptive_df = df[df['task'] == 'qa3']

# 일치 비율 계산 및 저장
qa_descriptive_df['token_match_score'] = qa_descriptive_df.apply(lambda row: token_overlap(row['output'], row['model_output']), axis=1)

# 평균 점수를 정확도로 간주
accuracy = qa_descriptive_df['token_match_score'].mean()

print(f"Token Match Accuracy: {accuracy:.2%}")


Token Match Accuracy: 0.00%


/tmp/ipykernel_2833459/841677399.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_descriptive_df['token_match_score'] = qa_descriptive_df.apply(lambda row: token_overlap(row['output'], row['model_output']), axis=1)


In [10]:
# nli

nli_df = df[df['task'] == 'nli']

correct_predictions = (nli_df['output'] == nli_df['model_output']).sum()
total_predictions = len(nli_df) 

nli_acc = correct_predictions / total_predictions

print(nli_acc)  # bioinstruct - 0.33 

0.24


In [11]:
# re

import pandas as pd

df_re = df[df['task'] == 're']

# Precision, Recall, F1 계산 함수 (전체)
def calculate_total_metrics(output_col, model_output_col):
    total_true_positive = 0
    total_false_positive = 0
    total_false_negative = 0

    for output, model_output in zip(output_col, model_output_col):
        # ','로 구분된 문자열을 집합으로 변환
        output_set = set(output.split(', '))
        model_output_set = set(model_output.split(', '))

        # 교집합, 정답의 크기, 모델 예측의 크기 계산
        true_positive = len(output_set & model_output_set)
        false_positive = len(model_output_set - output_set)
        false_negative = len(output_set - model_output_set)

        # 누적 합산
        total_true_positive += true_positive
        total_false_positive += false_positive
        total_false_negative += false_negative

    # 총 Precision, Recall, F1 계산
    precision = total_true_positive / (total_true_positive + total_false_positive) if (total_true_positive + total_false_positive) > 0 else 0
    recall = total_true_positive / (total_true_positive + total_false_negative) if (total_true_positive + total_false_negative) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

# 메트릭 계산
total_precision, total_recall, total_f1 = calculate_total_metrics(df_re['output'], df_re['model_output'])

# 결과 출력
print(f"Total Precision: {total_precision:.4f}")
print(f"Total Recall: {total_recall:.4f}")
print(f"Total F1-Score: {total_f1:.4f}")


Total Precision: 0.0000
Total Recall: 0.0000
Total F1-Score: 0.0000


In [12]:
# ie

from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

# 데이터프레임 예시 (df_ie 가 주어진 데이터프레임)
df_ie = df[df['task'] == 'ie']

df_ie["output"] = df_ie["output"].str.lower().str.split(", ")
df_ie["model_output"] = df_ie["model_output"].str.lower().str.split(", ")

# Precision, Recall, F1-score 계산 함수
def calculate_scores(y_true, y_pred):
    all_precisions = []
    all_recalls = []
    all_f1s = []
    
    for true_vals, pred_vals in zip(y_true, y_pred):
        true_set = set(true_vals) if isinstance(true_vals, list) else set()
        pred_set = set(pred_vals) if isinstance(pred_vals, list) else set()

        TP = len(true_set & pred_set)  # True Positives (정답과 예측이 일치하는 것)
        FP = len(pred_set - true_set)  # False Positives (예측했지만 정답이 아닌 것)
        FN = len(true_set - pred_set)  # False Negatives (정답이지만 예측하지 못한 것)

        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        all_precisions.append(precision)
        all_recalls.append(recall)
        all_f1s.append(f1)

    return sum(all_precisions) / len(all_precisions), sum(all_recalls) / len(all_recalls), sum(all_f1s) / len(all_f1s)

# Precision, Recall, F1-score 계산
precision, recall, f1 = calculate_scores(df_ie["output"], df_ie["model_output"])

# 결과 출력
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Precision: 0.1102
Recall: 0.1816
F1-score: 0.1255


/tmp/ipykernel_2833459/1779761820.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ie["output"] = df_ie["output"].str.lower().str.split(", ")
/tmp/ipykernel_2833459/1779761820.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ie["model_output"] = df_ie["model_output"].str.lower().str.split(", ")


In [13]:
# summarization
import openai
import pandas as pd
import re
from sklearn.metrics import f1_score
from bert_score import score as bert_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# BLEURT 모델 로드
bleurt_model_name = "Elron/bleurt-large-512"
tokenizer = AutoTokenizer.from_pretrained(bleurt_model_name)
bleurt_model = AutoModelForSequenceClassification.from_pretrained(bleurt_model_name)
bleurt_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bleurt_model = bleurt_model.to(device)

MAX_CONTEXT_LENGTH = 8192

# GPT-4 평가 함수
def evaluate_with_gpt4(input_text, model_output, true_output):
    input_text = input_text[:MAX_CONTEXT_LENGTH]
    model_output = model_output[:MAX_CONTEXT_LENGTH]
    true_output = true_output[:MAX_CONTEXT_LENGTH]

    prompt = f"""
    You are tasked with evaluating the quality of a QA model's responses based on the following metrics:
    1. **Coherence**: Does the model's response logically align with the context provided in the input?
    2. **Completeness**: Does the model's response sufficiently answer the question in the input?
    3. **Naturalness**: Does the model's response sound fluent and human-like?

    **Input**:
    {input_text}

    **Model's Response**:
    {model_output}

    **True Answer**:
    {true_output}

    Please rate each metric on a scale from 1 to 5. 
    Example response format:
    - Coherence: X.X
    - Completeness: X.X
    - Naturalness: X.X
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",
            messages=[
                {"role": "system", "content": "You are an expert evaluator for Summarization models."},
                {"role": "user", "content": prompt}
            ]
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print("Error with GPT-4 API:", e)
        return None

# GPT-4 점수 추출
def extract_scores(evaluation):
    if evaluation is None:
        return {"Coherence": 0.0, "Completeness": 0.0, "Naturalness": 0.0}
    coherence = re.search(r"Coherence: (\d\.\d)", evaluation)
    completeness = re.search(r"Completeness: (\d\.\d)", evaluation)
    naturalness = re.search(r"Naturalness: (\d\.\d)", evaluation)
    
    return {
        "Coherence": float(coherence.group(1)) if coherence else 0.0,
        "Completeness": float(completeness.group(1)) if completeness else 0.0,
        "Naturalness": float(naturalness.group(1)) if naturalness else 0.0
    }

# BLEURT 점수 계산
def calculate_bleurt(y_true, y_pred):
    inputs = tokenizer(y_pred, y_true, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        scores = bleurt_model(**inputs).logits

    if scores.numel() == 1:
        return float(scores.squeeze().item())  
    return [float(score) for score in scores.squeeze().tolist()]  

# BLEURT 및 BERTScore 계산
def calculate_bleurt_and_bertscore(y_true, y_pred):
    bleurt_score_value = calculate_bleurt(y_true, y_pred)
    _, _, bert_f1 = bert_score(y_pred, y_true, lang="en", rescale_with_baseline=True)
    bert_f1_avg = sum(bert_f1) / len(bert_f1) if len(bert_f1) > 0 else 0

    return {
        "BLEURT": bleurt_score_value if isinstance(bleurt_score_value, float) else sum(bleurt_score_value) / len(bleurt_score_value),
        "BERTScore_F1": bert_f1_avg
    }

# 점수 정규화 함수
def normalize_scores(df, column):
    if column not in df.columns:
        print(f"Warning: Column {column} not found in DataFrame. Skipping normalization.")
        return df
    df[column] = df[column].apply(lambda x: float(x) if isinstance(x, torch.Tensor) else x)
    min_val, max_val = df[column].min(), df[column].max()
    df[column] = df[column].apply(lambda x: (x - min_val) / (max_val - min_val) if max_val > min_val else 0.5)
    return df

qa_df = df[df['task'] == 'summarization']
results = []

for _, row in qa_df.iterrows():
    input_text, model_output, true_output = row['input'], row['model_output'], row['output']
    
    evaluation = evaluate_with_gpt4(input_text, model_output, true_output)
    print(f"Evaluation result:\n{evaluation}")
    
    scores = extract_scores(evaluation)
    metric_scores = calculate_bleurt_and_bertscore([true_output], [model_output])

    results.append({
        "input": input_text,
        "model_output": model_output,
        "true_output": true_output,
        "evaluation": evaluation,
        "Coherence": scores["Coherence"],
        "Completeness": scores["Completeness"],
        "Naturalness": scores["Naturalness"],
        "BLEURT": metric_scores.get("BLEURT", 0.0),  # 기본값 설정
        "BERTScore_F1": metric_scores.get("BERTScore_F1", 0.0)
    })

evaluation_df = pd.DataFrame(results)

# 'BLEURT' 컬럼이 존재하는지 확인 후 정규화 수행
evaluation_df = normalize_scores(evaluation_df, "BLEURT")
evaluation_df = normalize_scores(evaluation_df, "BERTScore_F1")

average_scores = evaluation_df[["Coherence", "Completeness", "Naturalness", "BLEURT", "BERTScore_F1"]].mean()
print("평균 점수:")
print(average_scores)


Evaluation result:
- Coherence: 2.0
- Completeness: 1.5
- Naturalness: 1.0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation result:
Based on the provided model's response and the true answer, here are the evaluations for each metric:

- Coherence: 4.5
- Completeness: 4.5
- Naturalness: 5.0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation result:
- Coherence: 5.0
- Completeness: 4.5
- Naturalness: 4.0


...

Evaluation result:
- Coherence: 4.5
- Completeness: 4.5
- Naturalness: 4.0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation result:
- Coherence: 4.5
- Completeness: 4.0
- Naturalness: 4.0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


평균 점수:
Coherence       4.440000
Completeness    4.020000
Naturalness     4.000000
BLEURT          0.337864
BERTScore_F1    0.759047
dtype: float64


In [14]:
# generation

import openai
import pandas as pd
import re
from sklearn.metrics import f1_score
from bert_score import score as bert_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# BLEURT 모델 로드
bleurt_model_name = "Elron/bleurt-large-512"
tokenizer = AutoTokenizer.from_pretrained(bleurt_model_name)
bleurt_model = AutoModelForSequenceClassification.from_pretrained(bleurt_model_name)
bleurt_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bleurt_model = bleurt_model.to(device)

MAX_CONTEXT_LENGTH = 8192

# GPT-4 평가 함수
def evaluate_with_gpt4(input_text, model_output, true_output):
    input_text = input_text[:MAX_CONTEXT_LENGTH]
    model_output = model_output[:MAX_CONTEXT_LENGTH]
    true_output = true_output[:MAX_CONTEXT_LENGTH]

    prompt = f"""
    You are tasked with evaluating the quality of a QA model's responses based on the following metrics:
    1. **Coherence**: Does the model's response logically align with the context provided in the input?
    2. **Completeness**: Does the model's response sufficiently answer the question in the input?
    3. **Naturalness**: Does the model's response sound fluent and human-like?

    **Input**:
    {input_text}

    **Model's Response**:
    {model_output}

    **True Answer**:
    {true_output}

    Please rate each metric on a scale from 1 to 5. 
    Example response format:
    - Coherence: X.X
    - Completeness: X.X
    - Naturalness: X.X
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",
            messages=[
                {"role": "system", "content": "You are an expert evaluator for Summarization models."},
                {"role": "user", "content": prompt}
            ]
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print("Error with GPT-4 API:", e)
        return None

# GPT-4 점수 추출
def extract_scores(evaluation):
    if evaluation is None:
        return {"Coherence": 0.0, "Completeness": 0.0, "Naturalness": 0.0}
    coherence = re.search(r"Coherence: (\d\.\d)", evaluation)
    completeness = re.search(r"Completeness: (\d\.\d)", evaluation)
    naturalness = re.search(r"Naturalness: (\d\.\d)", evaluation)
    
    return {
        "Coherence": float(coherence.group(1)) if coherence else 0.0,
        "Completeness": float(completeness.group(1)) if completeness else 0.0,
        "Naturalness": float(naturalness.group(1)) if naturalness else 0.0
    }

# BLEURT 점수 계산
def calculate_bleurt(y_true, y_pred):
    inputs = tokenizer(y_pred, y_true, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        scores = bleurt_model(**inputs).logits

    if scores.numel() == 1:
        return float(scores.squeeze().item())  
    return [float(score) for score in scores.squeeze().tolist()]  

# BLEURT 및 BERTScore 계산
def calculate_bleurt_and_bertscore(y_true, y_pred):
    bleurt_score_value = calculate_bleurt(y_true, y_pred)
    _, _, bert_f1 = bert_score(y_pred, y_true, lang="en", rescale_with_baseline=True)
    bert_f1_avg = sum(bert_f1) / len(bert_f1) if len(bert_f1) > 0 else 0

    return {
        "BLEURT": bleurt_score_value if isinstance(bleurt_score_value, float) else sum(bleurt_score_value) / len(bleurt_score_value),
        "BERTScore_F1": bert_f1_avg
    }

# 점수 정규화 함수
def normalize_scores(df, column):
    if column not in df.columns:
        print(f"Warning: Column {column} not found in DataFrame. Skipping normalization.")
        return df
    df[column] = df[column].apply(lambda x: float(x) if isinstance(x, torch.Tensor) else x)
    min_val, max_val = df[column].min(), df[column].max()
    df[column] = df[column].apply(lambda x: (x - min_val) / (max_val - min_val) if max_val > min_val else 0.5)
    return df

qa_df = df[df['task'] == 'generation']
results = []

for _, row in qa_df.iterrows():
    input_text, model_output, true_output = row['input'], row['model_output'], row['output']
    
    evaluation = evaluate_with_gpt4(input_text, model_output, true_output)
    print(f"Evaluation result:\n{evaluation}")
    
    scores = extract_scores(evaluation)
    metric_scores = calculate_bleurt_and_bertscore([true_output], [model_output])

    results.append({
        "input": input_text,
        "model_output": model_output,
        "true_output": true_output,
        "evaluation": evaluation,
        "Coherence": scores["Coherence"],
        "Completeness": scores["Completeness"],
        "Naturalness": scores["Naturalness"],
        "BLEURT": metric_scores.get("BLEURT", 0.0),  # 기본값 설정
        "BERTScore_F1": metric_scores.get("BERTScore_F1", 0.0)
    })

evaluation_df = pd.DataFrame(results)

# 'BLEURT' 컬럼이 존재하는지 확인 후 정규화 수행
evaluation_df = normalize_scores(evaluation_df, "BLEURT")
evaluation_df = normalize_scores(evaluation_df, "BERTScore_F1")

average_scores = evaluation_df[["Coherence", "Completeness", "Naturalness", "BLEURT", "BERTScore_F1"]].mean()
print("평균 점수:")
print(average_scores)


Evaluation result:
- Coherence: 4.0
- Completeness: 4.0
- Naturalness: 3.5


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation result:
- Coherence: 4.5
- Completeness: 4.0
- Naturalness: 4.0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation result:
- Coherence: 4.0
- Completeness: 5.0
- Naturalness: 4.5


...

Evaluation result:
- Coherence: 4.5
- Completeness: 4.5
- Naturalness: 4.0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation result:
- Coherence: 4.5
- Completeness: 4.0
- Naturalness: 3.5


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


평균 점수:
Coherence       3.440000
Completeness    3.180000
Naturalness     3.140000
BLEURT          0.545510
BERTScore_F1    0.884018
dtype: float64


In [20]:
# daily diets

import openai
import pandas as pd
import re
from tqdm import tqdm

from dotenv import load_dotenv
import os

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

MAX_CONTEXT_LENGTH = 8192

def evaluate_with_gpt4(input_text, model_output, true_output, task_type):
    input_text = input_text[:MAX_CONTEXT_LENGTH]
    model_output = model_output[:MAX_CONTEXT_LENGTH]
    true_output = true_output[:MAX_CONTEXT_LENGTH]

    if task_type == "daily_diets":
        prompt = f"""
        You are tasked with evaluating the quality of a meal recommendation model's responses based on the following metrics:
        1. **Coherence**: Does the model's response logically align with the context provided in the input?
        2. **Completeness**: Does the model's response sufficiently answer the input request?
        3. **Naturalness**: Does the model's response sound fluent and human-like?
        4. **Nutritional Adequacy**: Does the meal align with the nutritional goals in the input, considering reasonable flexibility and practical applicability in real-life scenarios?
        5. **Caloric Balance**: Are the recommended meals well-balanced in terms of calories?

        **Input**:
        {input_text}

        **Model's Response**:
        {model_output}

        **True Answer**:
        {true_output}

        Please rate each metric on a scale from 1 to 5. 
        Example response format:
        - Coherence: X.X
        - Completeness: X.X
        - Naturalness: X.X
        - Nutritional Adequacy: X.X
        - Caloric Balance: X.X
        """
    elif task_type == "alternative_diets":
        prompt = f"""
        You are tasked with evaluating the quality of a meal recommendation model's responses based on the following metrics:
        1. **Coherence**: Does the model's response logically align with the context provided in the input?
        2. **Completeness**: Does the model's response sufficiently answer the input request?
        3. **Naturalness**: Does the model's response sound fluent and human-like?
        4. **Improvement**: Does the recommended meal address the shortcomings of the previous meal?
        5. **Suitability**: Is the recommended meal suitable for a diabetes patient?

        **Input**:
        {input_text}

        **Model's Response**:
        {model_output}

        **True Answer**:
        {true_output}

        Please rate each metric on a scale from 1 to 5. 
        Example response format:
        - Coherence: X.X
        - Completeness: X.X
        - Naturalness: X.X
        - Improvement: X.X
        - Suitability: X.X
        """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",
            messages=[
                {"role": "system", "content": "You are an expert evaluator for meal recommendation models."},
                {"role": "user", "content": prompt}
            ]
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print("Error with GPT-4 API:", e)
        return None

# 점수 추출 함수
def extract_scores(evaluation, task_type):
    if evaluation is None:
        if task_type == "daily_diets":
            return {
                "Coherence": 0.0,
                "Completeness": 0.0,
                "Naturalness": 0.0,
                "Nutritional Adequacy": 0.0,
                "Caloric Balance": 0.0
            }
        elif task_type == "alternative_diets":
            return {
                "Coherence": 0.0,
                "Completeness": 0.0,
                "Naturalness": 0.0,
                "Improvement": 0.0,
                "Suitability": 0.0
            }

    scores = {}
    if task_type == "daily_diets":
        metrics = ["Coherence", "Completeness", "Naturalness", "Nutritional Adequacy", "Caloric Balance"]
    elif task_type == "alternative_diets":
        metrics = ["Coherence", "Completeness", "Naturalness", "Improvement", "Suitability"]

    for metric in metrics:
        match = re.search(fr"{metric}: (\d\.\d)", evaluation)
        scores[metric] = float(match.group(1)) if match else 0.0

    return scores

results = []

daily_df = df[(df['task'] == 'daily_diets') & (df['output'].str.contains('"Breakfast"'))]

for _, row in tqdm(daily_df.iterrows(), total=len(daily_df), desc="Evaluating daily diets"):
    evaluation = evaluate_with_gpt4(row['input'], row['model_output'], row['output'], "daily_diets")
    scores = extract_scores(evaluation, "daily_diets")
    results.append({**row.to_dict(), **scores})

alternative_df = df[df['task'] == 'alternative_diets']

for _, row in tqdm(alternative_df.iterrows(), total=len(alternative_df), desc="Evaluating alternative diets"):
    evaluation = evaluate_with_gpt4(row['input'], row['model_output'], row['output'], "alternative_diets")
    scores = extract_scores(evaluation, "alternative_diets")
    results.append({**row.to_dict(), **scores})

evaluation_df = pd.DataFrame(results)

if "Nutritional Adequacy" in evaluation_df.columns and "Caloric Balance" in evaluation_df.columns:
    daily_avg = evaluation_df[evaluation_df['task'] == 'daily_diets'][[
        "Coherence", "Completeness", "Naturalness", 
        "Nutritional Adequacy", "Caloric Balance"
    ]].mean()
    print("Daily Diets Average Scores:")
    print(daily_avg)

if "Improvement" in evaluation_df.columns and "Suitability" in evaluation_df.columns:
    alternative_avg = evaluation_df[evaluation_df['task'] == 'alternative_diets'][[
        "Coherence", "Completeness", "Naturalness", 
        "Improvement", "Suitability"
    ]].mean()
    print("\nAlternative Diets Average Scores:")
    print(alternative_avg)
else:
    missing_columns = [col for col in ["Improvement", "Suitability"] if col not in evaluation_df.columns]
    print("\nAlternative Diets scores not available:")
    print(f"Missing columns: {missing_columns}")


Evaluating daily diets: 100%|██████████| 21/21 [00:20<00:00,  1.03it/s]
Evaluating alternative diets: 0it [00:00, ?it/s]

Daily Diets Average Scores:
Coherence               4.166667
Completeness            3.285714
Naturalness             3.752381
Nutritional Adequacy    3.628571
Caloric Balance         3.838095
dtype: float64

Alternative Diets scores not available:
Missing columns: ['Improvement', 'Suitability']


In [16]:
# alternative diets

import openai
import pandas as pd
import re
from tqdm import tqdm

from dotenv import load_dotenv
import os

load_dotenv()


openai.api_key = os.getenv("OPENAI_API_KEY")

MAX_CONTEXT_LENGTH = 8192

def evaluate_with_gpt4(input_text, model_output, true_output, task_type):

    input_text = input_text[:MAX_CONTEXT_LENGTH]
    model_output = model_output[:MAX_CONTEXT_LENGTH]
    true_output = true_output[:MAX_CONTEXT_LENGTH]

    if task_type == "daily_diets":
        prompt = f"""
        You are tasked with evaluating the quality of a meal recommendation model's responses based on the following metrics:
        1. **Coherence**: Does the model's response logically align with the context provided in the input?
        2. **Completeness**: Does the model's response sufficiently answer the input request?
        3. **Naturalness**: Does the model's response sound fluent and human-like?
        4. **Nutritional Adequacy**: Does the meal response meet the nutritional goals mentioned in the input?
        5. **Caloric Balance**: Are the recommended meals well-balanced in terms of calories?

        **Input**:
        {input_text}

        **Model's Response**:
        {model_output}

        **True Answer**:
        {true_output}

        Please rate each metric on a scale from 1 to 5. 
        Example response format:
        - Coherence: X.X
        - Completeness: X.X
        - Naturalness: X.X
        - Nutritional Adequacy: X.X
        - Caloric Balance: X.X
        """
    elif task_type == "alternative_diet":
        prompt = f"""
        You are tasked with evaluating the quality of a meal recommendation model's responses based on the following metrics:

        1. **Coherence**: Does the model's response logically align with the context provided in the input?
        2. **Completeness**: Does the model's response sufficiently answer the input request?
        3. **Naturalness**: Does the model's response sound fluent and human-like?
        4. **Improvement**: Assume that the recommended alternative meal is an improvement over the previous meal. Evaluate how effectively it builds upon and enhances the previous meal, even if the changes are small or subtle.
        5. **Suitability**: Is the recommended meal suitable for a diabetes patient?

        **Input**:
        {input_text}

        **Model's Response**:
        {model_output}

        **True Answer**:
        {true_output}

        Please rate each metric on a scale from 1 to 5. 
        Example response format:
        - Coherence: X.X
        - Completeness: X.X
        - Naturalness: X.X
        - Improvement: X.X
        - Suitability: X.X
        """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",
            messages=[
                {"role": "system", "content": "You are an expert evaluator for meal recommendation models."},
                {"role": "user", "content": prompt}
            ]
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print("Error with GPT-4 API:", e)
        return None

def extract_scores(evaluation, task_type):
    if evaluation is None:
        if task_type == "daily_diets":
            return {
                "Coherence": 0.0,
                "Completeness": 0.0,
                "Naturalness": 0.0,
                "Nutritional Adequacy": 0.0,
                "Caloric Balance": 0.0
            }
        elif task_type == "alternative_diet":
            return {
                "Coherence": 0.0,
                "Completeness": 0.0,
                "Naturalness": 0.0,
                "Improvement": 0.0,
                "Suitability": 0.0
            }

    scores = {}
    if task_type == "daily_diets":
        metrics = ["Coherence", "Completeness", "Naturalness", "Nutritional Adequacy", "Caloric Balance"]
    elif task_type == "alternative_diet":
        metrics = ["Coherence", "Completeness", "Naturalness", "Improvement", "Suitability"]

    for metric in metrics:
        match = re.search(fr"{metric}: (\d\.\d)", evaluation)
        scores[metric] = float(match.group(1)) if match else 0.0

    return scores

results = []

alternative_df = df[df['task'] == 'alternative_diet']

for _, row in tqdm(alternative_df.iterrows(), total=len(alternative_df), desc="Evaluating alternative diets"):
    evaluation = evaluate_with_gpt4(row['input'], row['model_output'], row['output'], "alternative_diet")
    scores = extract_scores(evaluation, "alternative_diet")
    results.append({**row.to_dict(), **scores})

evaluation_df = pd.DataFrame(results)

if all(col in evaluation_df.columns for col in ["Improvement", "Suitability"]):
    alternative_avg = evaluation_df[[
        "Coherence", "Completeness", "Naturalness", 
        "Improvement", "Suitability"
    ]].mean()
    print("\nAlternative Diets Average Scores:")
    print(alternative_avg)


Evaluating alternative diets: 100%|██████████| 25/25 [00:22<00:00,  1.12it/s]


Alternative Diets Average Scores:
Coherence       4.14
Completeness    3.90
Naturalness     3.88
Improvement     3.72
Suitability     4.14
dtype: float64


In [17]:
import pandas as pd

dfh = pd.read_csv("/data/jaesung/llm_for_diabetes/src/data/data2_daily_diets/diabetes_food_hub_new_nutri_facts.csv")
dfh.head(2)

,title,description,prep_time,cook_time,servings,steps,tags,nutrition_facts,ingredients
0,Raspberry Swirl Frozen Yogurt Bark,Raspberry Swirl Frozen Yogurt Bark: Dive into ...,10 min,4 hr,6 Servings,['Cover a freezer-safe tray with parchment pap...,"['Kid Friendly', 'Vegetarian', 'Dessert', 'Sna...","{'Servings': '6 Servings', 'Serving Size': '1 ...","[{'label': 'Plain Nonfat Greek yogurt', 'us_me..."
1,Maple-Pumpkin Spice Oatmeal Cookies,Description not found,10 min,25 min,14 Servings,['Preheat the oven to 350 degrees F. Line two ...,"['Kid Friendly', 'Vegetarian', 'Snacks', 'Glut...","{'Servings': '14 Servings', 'Serving Size': '1...","[{'label': 'old-fashioned rolled oats', 'us_me..."


In [18]:
# daily diet - nutri score

import ast
import pandas as pd
import json
import re
import openai
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from dotenv import load_dotenv
import os
from tqdm import tqdm

# Load environment variables
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_numeric_value(value):
    try:
        if isinstance(value, str):
            match = re.search(r"(\d+(\.\d+)?)", value)
            if match:
                return float(match.group(1))
        elif isinstance(value, (int, float)):
            return float(value)
    except Exception as e:
        print(f"Error in extract_numeric_value: {e}, value: {value}")
    return 0.0

def is_valid_meal_structure(json_string):
    try:
        data = json.loads(json_string)
        return all(key in data for key in ['Breakfast', 'Lunch', 'Dinner'])
    except (json.JSONDecodeError, TypeError):
        return False

def find_most_similar_row(title, dfh):
    try:
        dfh['title'] = dfh['title'].fillna('')  # Handle NaN values
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(dfh['title'])
        input_vector = vectorizer.transform([title])
        similarities = cosine_similarity(input_vector, tfidf_matrix)
        most_similar_idx = similarities.argmax()
        return dfh.iloc[most_similar_idx]
    except Exception as e:
        print(f"Error in find_most_similar_row: {e}, title: {title}")
        return None

def identify_fruit_veg(ingredients_list):
    try:
        prompt = f"Identify which items in the following ingredient list are fruits or vegetables:\n\n{ingredients_list}\n\nReturn only the names of items that are fruits or vegetables in a Python list format."
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an assistant identifying fruits and vegetables."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,
            temperature=0
        )
        fruits_vegetables = response['choices'][0]['message']['content']
        return ast.literal_eval(fruits_vegetables)
    except Exception as e:
        print(f"Error identifying fruits and vegetables: {e}")
        return []

def calculate_fruit_veg_points(ingredients, total_weight):
    try:
        ingredients_list = ast.literal_eval(ingredients)
        fruit_veg_labels = identify_fruit_veg(ingredients_list)

        fruit_veg_weight = 0
        for ingredient in ingredients_list:
            label = ingredient.get('label', '')
            weight = extract_numeric_value(ingredient.get('metric_measure', 0))
            if label in fruit_veg_labels:
                fruit_veg_weight += weight

        # 과일/채소 비율을 100g 기준으로 변환
        fruit_veg_ratio = (fruit_veg_weight / total_weight) * 100 if total_weight > 0 else 0

        if fruit_veg_ratio > 80:
            return 5
        elif fruit_veg_ratio > 60:
            return 2
        elif fruit_veg_ratio > 40:
            return 1
        else:
            return 0
    except Exception as e:
        print(f"Error calculating fruit_veg_points: {e}")
        return 0

def extract_nested_value(data, keys, default=0):
    try:
        for key in keys:
            if isinstance(data, dict):
                data = data.get(key, {})
            else:
                return default
        return extract_numeric_value(data) if isinstance(data, (int, float, str)) else default
    except Exception as e:
        print(f"Error in extract_nested_value: {e}, keys: {keys}, data: {data}")
        return default

def calculate_nutri_score(nutrition_facts, ingredients):
    try:
        if isinstance(nutrition_facts, str):
            nutrition_facts = ast.literal_eval(nutrition_facts)

        # 전체 무게 계산
        total_weight = sum(
            extract_numeric_value(ingredient.get('metric_measure', 0)) 
            for ingredient in ast.literal_eval(ingredients)
        )
        if total_weight == 0:
            print("Warning: Total weight is zero. Skipping calculation.")
            return None

        # 100g 기준으로 성분 정규화
        energy = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Calories']) / total_weight * 100
        saturated_fat = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Total Fat', 'Amount']) / total_weight * 100
        sugar = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Total Carbohydrates', 'Total Sugars']) / total_weight * 100
        sodium = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Sodium']) / total_weight * 100
        fiber = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Total Carbohydrates', 'Dietary Fiber']) / total_weight * 100
        protein = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Protein']) / total_weight * 100

        # Unfavorable points calculation
        energy_points = min(energy / 80, 800)
        saturated_fat_points = min(saturated_fat / 1, 10)
        sugar_points = min(sugar / 4.5, 45)
        sodium_points = min(sodium / 90, 900)

        unfavorable_points = energy_points + saturated_fat_points + sugar_points + sodium_points

        # Favorable points calculation
        fiber_points = min(fiber / 0.7, 3.5)
        protein_points = min(protein / 1.6, 8.0)
        fruit_veg_points = calculate_fruit_veg_points(ingredients, total_weight)

        favorable_points = fiber_points + protein_points + fruit_veg_points

        # Final Nutri-Score calculation
        total_score = unfavorable_points - favorable_points
        return total_score
    except Exception as e:
        print(f"Error in calculate_nutri_score: {e}, nutrition_facts: {nutrition_facts}")
        return None

def get_nutri_score_grade(score):
    if score <= -1:
        return "A"
    elif score <= 2:
        return "B"
    elif score <= 10:
        return "C"
    elif score <= 18:
        return "D"
    else:
        return "E"

def calculate_meal_nutri_score(meal_data, dfh):
    meal_scores = {}

    for meal, title in meal_data.items():
        matched_row = find_most_similar_row(title, dfh)
        if matched_row is None:
            continue

        nutrition_facts = matched_row['nutrition_facts']
        ingredients = matched_row['ingredients']
        score = calculate_nutri_score(nutrition_facts, ingredients)

        if score is None:
            print(f"Warning: Nutri-Score calculation failed for meal '{meal}' with title '{title}'.")
            grade = "N/A"
        else:
            grade = get_nutri_score_grade(score)

        meal_scores[meal] = {'score': score, 'grade': grade}

    return meal_scores

def calculate_scores_with_comparison(df, dfh):
    results = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        output_scores = {}
        model_scores = {}
        if is_valid_meal_structure(row.get('output', '')):
            output_data = json.loads(row['output'])
            output_scores = calculate_meal_nutri_score(output_data, dfh)
        if is_valid_meal_structure(row.get('model_output', '')):
            model_data = json.loads(row['model_output'])
            model_scores = calculate_meal_nutri_score(model_data, dfh)
        results.append({'row_index': idx, 'output_scores': output_scores, 'model_scores': model_scores})
    return results

def calculate_average_scores(results):
    """
    Calculate the average Nutri-Scores for outputs and model outputs.
    """
    output_total_score = 0
    model_total_score = 0
    output_count = 0
    model_count = 0

    for result in results:
        # Extract output scores
        for meal, score_data in result['output_scores'].items():
            if score_data['score'] is not None:
                output_total_score += score_data['score']
                output_count += 1

        # Extract model scores
        for meal, score_data in result['model_scores'].items():
            if score_data['score'] is not None:
                model_total_score += score_data['score']
                model_count += 1

    # Calculate averages
    output_avg = output_total_score / output_count if output_count > 0 else None
    model_avg = model_total_score / model_count if model_count > 0 else None

    return output_avg, model_avg


# 'daily_diets' task Nutri-Score calculation
filtered_df = df[df['task'] == 'daily_diets']
results = calculate_scores_with_comparison(filtered_df, dfh)

# Calculate overall averages
output_avg, model_avg = calculate_average_scores(results)

# Print results
print("=== Results for Each Row ===")
for result in results:
    print(f"Row Index: {result['row_index']}")
    print(f"Output Scores: {result['output_scores']}")
    print(f"Model Output Scores: {result['model_scores']}")
    print()

print("=== Overall Averages ===")
print(f"Output Average Nutri-Score: {output_avg}")
print(f"Model Output Average Nutri-Score: {model_avg}")


  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:37<00:00,  1.51s/it]

=== Results for Each Row ===
Row Index: 0
Output Scores: {'Breakfast': {'score': 1.2832821300563242, 'grade': 'B'}, 'Lunch': {'score': 0.12827070932539675, 'grade': 'B'}, 'Dinner': {'score': 0.1844070961718014, 'grade': 'B'}}
Model Output Scores: {}

Row Index: 1
Output Scores: {'Breakfast': {'score': 0.27903677232536306, 'grade': 'B'}, 'Lunch': {'score': 1.2434792311769467, 'grade': 'B'}, 'Dinner': {'score': 0.12827070932539675, 'grade': 'B'}}
Model Output Scores: {}

Row Index: 2
Output Scores: {'Breakfast': {'score': -0.06054961667206582, 'grade': 'B'}, 'Lunch': {'score': 1.4849885674520331, 'grade': 'B'}, 'Dinner': {'score': 0.12827070932539675, 'grade': 'B'}}
Model Output Scores: {}

Row Index: 3
Output Scores: {'Breakfast': {'score': -5.211711711711713, 'grade': 'A'}, 'Lunch': {'score': -0.6524003743136113, 'grade': 'B'}, 'Dinner': {'score': 1.210600194254846, 'grade': 'B'}}
Model Output Scores: {}

Row Index: 4
Output Scores: {'Breakfast': {'score': 0.08622999129328224, 'grade':

In [19]:
# alternative diet - nutri score

import ast
import pandas as pd
import json
import re
import openai
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from dotenv import load_dotenv
import os
from tqdm import tqdm

# Load environment variables
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_numeric_value(value):
    try:
        if isinstance(value, str):
            match = re.search(r"(\d+(\.\d+)?)", value)
            if match:
                return float(match.group(1))
        elif isinstance(value, (int, float)):
            return float(value)
    except Exception as e:
        print(f"Error in extract_numeric_value: {e}, value: {value}")
    return 0.0

def is_valid_meal_structure(json_string):
    try:
        data = json.loads(json_string)
        return isinstance(data, dict)
    except (json.JSONDecodeError, TypeError):
        return False

def find_most_similar_row(title, dfh):
    try:
        dfh['title'] = dfh['title'].fillna('')  # Handle NaN values
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(dfh['title'])
        input_vector = vectorizer.transform([title])
        similarities = cosine_similarity(input_vector, tfidf_matrix)
        most_similar_idx = similarities.argmax()
        return dfh.iloc[most_similar_idx]
    except Exception as e:
        print(f"Error in find_most_similar_row: {e}, title: {title}")
        return None

def identify_fruit_veg(ingredients_list):
    try:
        prompt = f"Identify which items in the following ingredient list are fruits or vegetables:\n\n{ingredients_list}\n\nReturn only the names of items that are fruits or vegetables in a Python list format."
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an assistant identifying fruits and vegetables."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,
            temperature=0
        )
        fruits_vegetables = response['choices'][0]['message']['content']
        return ast.literal_eval(fruits_vegetables)
    except Exception as e:
        print(f"Error identifying fruits and vegetables: {e}")
        return []

def calculate_fruit_veg_points(ingredients, total_weight):
    try:
        ingredients_list = ast.literal_eval(ingredients)
        fruit_veg_labels = identify_fruit_veg(ingredients_list)

        fruit_veg_weight = 0
        for ingredient in ingredients_list:
            label = ingredient.get('label', '')
            weight = extract_numeric_value(ingredient.get('metric_measure', 0))
            if label in fruit_veg_labels:
                fruit_veg_weight += weight

        # 과일/채소 비율을 100g 기준으로 변환
        fruit_veg_ratio = (fruit_veg_weight / total_weight) * 100 if total_weight > 0 else 0

        if fruit_veg_ratio > 80:
            return 5
        elif fruit_veg_ratio > 60:
            return 2
        elif fruit_veg_ratio > 40:
            return 1
        else:
            return 0
    except Exception as e:
        print(f"Error calculating fruit_veg_points: {e}")
        return 0

def extract_nested_value(data, keys, default=0):
    try:
        for key in keys:
            if isinstance(data, dict):
                data = data.get(key, {})
            else:
                return default
        return extract_numeric_value(data)
    except Exception as e:
        print(f"Error in extract_nested_value: {e}, keys: {keys}, data: {data}")
        return default

def calculate_nutri_score(nutrition_facts, ingredients):
    try:
        if isinstance(nutrition_facts, str):
            nutrition_facts = ast.literal_eval(nutrition_facts)

        # 전체 무게 계산
        total_weight = sum(
            extract_numeric_value(ingredient.get('metric_measure', 0)) 
            for ingredient in ast.literal_eval(ingredients)
        )
        if total_weight == 0:
            print("Warning: Total weight is zero. Skipping calculation.")
            return None

        # 100g 기준으로 성분 정규화
        energy = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Calories']) / total_weight * 100
        saturated_fat = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Total Fat', 'Amount']) / total_weight * 100
        sugar = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Total Carbohydrates', 'Total Sugars']) / total_weight * 100
        sodium = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Sodium']) / total_weight * 100
        fiber = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Total Carbohydrates', 'Dietary Fiber']) / total_weight * 100
        protein = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Protein']) / total_weight * 100

        # Unfavorable points calculation
        energy_points = min(energy / 80, 800)
        saturated_fat_points = min(saturated_fat / 1, 10)
        sugar_points = min(sugar / 4.5, 45)
        sodium_points = min(sodium / 90, 900)

        unfavorable_points = energy_points + saturated_fat_points + sugar_points + sodium_points

        # Favorable points calculation
        fiber_points = min(fiber / 0.7, 3.5)
        protein_points = min(protein / 1.6, 8.0)
        fruit_veg_points = calculate_fruit_veg_points(ingredients, total_weight)

        favorable_points = fiber_points + protein_points + fruit_veg_points

        # Final Nutri-Score calculation
        total_score = unfavorable_points - favorable_points
        return total_score
    except Exception as e:
        print(f"Error in calculate_nutri_score: {e}, nutrition_facts: {nutrition_facts}")
        return None

def get_nutri_score_grade(score):
    if score <= -1:
        return "A"
    elif score <= 2:
        return "B"
    elif score <= 10:
        return "C"
    elif score <= 18:
        return "D"
    else:
        return "E"

def calculate_scores_with_comparison_no_meals(df, dfh):
    results = []
    output_scores_list = []
    model_output_scores_list = []

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        try:
            output_text = row.get('output', '')
            if output_text:
                matched_row = find_most_similar_row(output_text, dfh)
                if matched_row is not None:
                    nutrition_facts = matched_row['nutrition_facts']
                    ingredients = matched_row['ingredients']
                    output_score = calculate_nutri_score(nutrition_facts, ingredients)
                    output_scores_list.append(output_score)
                else:
                    output_score = None

            model_output_text = row.get('model_output', '')
            if model_output_text:
                matched_row = find_most_similar_row(model_output_text, dfh)
                if matched_row is not None:
                    nutrition_facts = matched_row['nutrition_facts']
                    ingredients = matched_row['ingredients']
                    model_output_score = calculate_nutri_score(nutrition_facts, ingredients)
                    model_output_scores_list.append(model_output_score)
                else:
                    model_output_score = None

            results.append({
                'row_index': idx,
                'output_score': output_score,
                'model_output_score': model_output_score
            })

        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            results.append({
                'row_index': idx,
                'output_score': None,
                'model_output_score': None
            })

    final_output_avg = sum(output_scores_list) / len(output_scores_list) if output_scores_list else None
    final_model_output_avg = sum(model_output_scores_list) / len(model_output_scores_list) if model_output_scores_list else None

    print(f"Output Average Nutri-Score: {final_output_avg}")
    print(f"Model Output Average Nutri-Score: {final_model_output_avg}")

    return results

# Execution
filtered_df = df[df['task'] == 'alternative_diet']
results = calculate_scores_with_comparison_no_meals(filtered_df, dfh)

# Print results
for result in results:
    print(result)


100%|██████████| 25/25 [00:31<00:00,  1.26s/it]

Output Average Nutri-Score: 0.4176467924666062
Model Output Average Nutri-Score: 0.592826207957158
{'row_index': 50, 'output_score': -0.5456349206349205, 'model_output_score': 0.5184079415151972}
{'row_index': 51, 'output_score': 0.32153783832762994, 'model_output_score': 1.1896494708994707}
{'row_index': 52, 'output_score': 1.2495974235104654, 'model_output_score': 0.5184079415151972}
{'row_index': 53, 'output_score': -0.2137724271482232, 'model_output_score': -0.9043691424237499}
{'row_index': 54, 'output_score': 0.09538432905267075, 'model_output_score': 2.305605786618444}
{'row_index': 55, 'output_score': 0.09538432905267075, 'model_output_score': 8.479166666666668}
{'row_index': 56, 'output_score': -4.595731219412345, 'model_output_score': 0.5184079415151972}
{'row_index': 57, 'output_score': 0.32153783832762994, 'model_output_score': 10.671052631578949}
{'row_index': 58, 'output_score': -0.04938140184041795, 'model_output_score': 0.5184079415151972}
{'row_index': 59, 'output_scor